In [1]:
import pandas as pd
import numpy as np

from datetime import datetime
from datetime import date
from elasticsearch import Elasticsearch

In [60]:
es = Elasticsearch(
    cloud_id="elasticsearch:dXMtY2VudHJhbDEuZ2NwLmNsb3VkLmVzLmlvJDc5ZWZiYTgzYWZiNjQzZDliODVhYjViMWY5MTBmOTc0JDgzYmFiM2M3MWM0NzQxMzc4MzEzOTk3OWI4OTMyZWUx",
http_auth=('elastic', 'WONF3FVvSUJ5g3vLyMn9jcfE'))
#scheme="https", port=443,) 
es.info()

{'name': 'instance-0000000000',
 'cluster_name': '79efba83afb643d9b85ab5b1f910f974',
 'cluster_uuid': 'cCyF0494RESieWo_5xYQaQ',
 'version': {'number': '7.8.0',
  'build_flavor': 'default',
  'build_type': 'docker',
  'build_hash': '757314695644ea9a1dc2fecd26d1a43856725e65',
  'build_date': '2020-06-14T19:35:50.234439Z',
  'build_snapshot': False,
  'lucene_version': '8.5.1',
  'minimum_wire_compatibility_version': '6.8.0',
  'minimum_index_compatibility_version': '6.0.0-beta1'},
 'tagline': 'You Know, for Search'}

In [61]:
from elasticsearch.client import MlClient

In [62]:
elastic_jobs=['bucketspan_1d_p','bucketspan_7d_p']
tablas={}
for j in range(0,2):
    tablas[elastic_jobs[j]]=MlClient(es).get_records(job_id=str(elastic_jobs[j]),size=10000, record_score=75)
tablas

{'bucketspan_1d_p': {'count': 1227,
  'records': [{'job_id': 'bucketspan_1d_p',
    'result_type': 'record',
    'probability': 8.421908784064792e-156,
    'multi_bucket_impact': -5.0,
    'record_score': 99.93095288720326,
    'initial_record_score': 99.93095288720326,
    'bucket_span': 86400,
    'detector_index': 1,
    'is_interim': False,
    'timestamp': 1568073600000,
    'partition_field_name': 'asset_id',
    'partition_field_value': 'asset_asset_asset_7',
    'function': 'high_mean',
    'function_description': 'mean',
    'typical': [0.013359308242797852],
    'actual': [13.0],
    'field_name': 't2',
    'influencers': [{'influencer_field_name': 'asset_id',
      'influencer_field_values': ['asset_asset_asset_7']}],
    'asset_id': ['asset_asset_asset_7']},
   {'job_id': 'bucketspan_1d_p',
    'result_type': 'record',
    'probability': 8.367742026504717e-156,
    'multi_bucket_impact': -5.0,
    'record_score': 99.93072665229477,
    'initial_record_score': 99.93072665229

In [63]:
for key, value in tablas.items():
    pd.DataFrame(value['records']).to_csv('Output/df_elastic_'+str(key)+'.csv')

##  Formatear datos de los 2 archivos  guardados

In [7]:
def timestamp_to_date(x): 
    ''' Funcion para transformar la variable timestamp obtenida de Elastic a fecha.
    x:timestamp de elastic
    Se emplea la función directa de Python en vez de utilizar la funcion Timestamp() de Pandas, porque la segunda ocasiona
    un desfase de un día.
    Se divide entre 1000 porque fromtimestamp espera recibir segundos y no milisegundos, como devuelve Elastic timestamp.
    '''
    return date.fromtimestamp(x/1000)  

In [10]:
X_train=pd.read_csv("Output/X_train_formateado.csv")
#Formateamos las columnas que serán clave para la union de tablas
X_train['asset_id']=X_train['asset_id'].astype(object)
X_train['date']=pd.to_datetime(X_train['date']) 
X_train['date']=X_train['date'].dt.strftime('%Y-%m-%d')
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60424 entries, 0 to 60423
Data columns (total 12 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   asset_id  60424 non-null  object 
 1   group_id  60424 non-null  int64  
 2   week      60424 non-null  int64  
 3   weekday   60424 non-null  int64  
 4   t1        24752 non-null  float64
 5   t2        24752 non-null  float64
 6   t3        60424 non-null  float64
 7   t4        60424 non-null  float64
 8   total_t   60424 non-null  int64  
 9   week_day  60424 non-null  object 
 10  date      60424 non-null  object 
 11  day       60424 non-null  int64  
dtypes: float64(4), int64(5), object(3)
memory usage: 5.5+ MB


In [13]:
X_train

,asset_id,group_id,week,weekday,t1,t2,t3,t4,total_t,week_day,date,day
0,1,2,1,1,NaN,NaN,19.0,0.0,2,1_1,2018-01-01,1
1,1,2,1,2,NaN,NaN,8.0,0.0,2,1_2,2018-01-02,2
2,1,2,1,3,NaN,NaN,9.0,0.0,2,1_3,2018-01-03,3
3,1,2,1,4,NaN,NaN,6.0,0.0,2,1_4,2018-01-04,4
4,1,2,1,5,NaN,NaN,6.0,2.0,2,1_5,2018-01-05,5
...,...,...,...,...,...,...,...,...,...,...,...,...
60419,83,3,104,3,1.0,0.0,48.0,8.0,4,104_3,2019-12-25,724
60420,83,3,104,4,0.0,0.0,37.0,1.0,4,104_4,2019-12-26,725
60421,83,3,104,5,1.0,0.0,48.0,9.0,4,104_5,2019-12-27,726
60422,83,3,104,6,2.0,0.0,53.0,11.0,4,104_6,2019-12-28,727


In [12]:
bucketspan=['1d_p', '7d_p']
col_de_interes=['job_id','multi_bucket_impact','record_score', 'typical', 'actual','field_name','asset_id','date']
col=['typical','actual','asset_id']

df={}
for bs in bucketspan:   #ciclo para transformar los datos descargados de elastic y guardados como csv para 1d bucket span y 7d
    
    #se leen los dataframe descargados de elastic 
    df['anomalias_'+bs]=pd.read_csv('Output/df_elastic_bucketspan_'+bs+'.csv')
    
    #se transforma el timestamp a fecha
    df['anomalias_'+bs]['date']=df['anomalias_'+bs]['timestamp'].apply(timestamp_to_date)
    df['anomalias_'+bs]['date']=pd.to_datetime(df['anomalias_'+bs]['date'])   
    
    # nos quedamos solamente con las variables que nos interesan
    df['anomalias_'+bs]=df['anomalias_'+bs][col_de_interes]
    df['anomalias_'+bs].columns=['bucketspan', 'multi_bucket_impact', 'record_score', 'typical', 'actual',
       'field_name', 'asset_id', 'date']

    #adecuamos el formato de las columnas
    for c in col:
        df['anomalias_'+bs][c]=df['anomalias_'+bs][c].str.strip('[]')
    
    df['anomalias_'+bs]['asset_id']=df['anomalias_'+bs]['asset_id'].str.strip('\'asset_')
    df['anomalias_'+bs]['bucketspan']=df['anomalias_'+bs]['bucketspan'].str.strip('bucketspan_')
    df['anomalias_'+bs]['asset_id']=pd.to_numeric(df['anomalias_'+bs]['asset_id'])
        
    #Filtramos las anomalias que se consideraran 'single bucketspan'
    df['anomalias_'+bs]=df['anomalias_'+bs][df['anomalias_'+bs]['multi_bucket_impact']<0]  
    
    #adecuamos el formato para hacerlo compatible con X_train_formateado.csv
    df['anomalias_'+bs]=pd.pivot_table(df['anomalias_'+bs],index=['asset_id','date'], columns=['field_name'], values=['record_score']).reset_index()
    df['anomalias_'+bs].columns=['asset_id','date','outlier_t1','outlier_t2','outlier_t3','outlier_t4']
    
    df['anomalias_'+bs]['asset_id']=df['anomalias_'+bs]['asset_id'].astype(object)
        
    pd.DataFrame(df['anomalias_'+bs]).to_csv('Output/df_outliers_Elastic_bucketspan_'+bs+'.csv',index=False)
    
#    df['anomalias_'+bs]=pd.merge(X_train,df['anomalias_'+bs],how='left', on=['asset_id','date'])

#    for m in ['t1','t2','t3','t4']:
#        df['anomalias_'+bs]['outlier_'+m]=df['anomalias_'+bs]['outlier_'+m].fillna(0)
    
#    for t in ['t1','t2']:
#        df['anomalias_'+bs]['outlier_'+t]=np.where(df['anomalias_'+bs]['total_t']==2,np.nan,df['anomalias_'+bs]['outlier_'+t])
        
df['anomalias_1d_p']

,asset_id,date,outlier_t1,outlier_t2,outlier_t3,outlier_t4
0,2,2018-10-26,NaN,NaN,NaN,77.24560
1,2,2018-10-27,NaN,NaN,NaN,80.20610
2,2,2019-01-11,NaN,NaN,NaN,77.91047
3,2,2019-01-15,NaN,NaN,NaN,76.13505
4,2,2019-01-20,NaN,NaN,NaN,78.59278
...,...,...,...,...,...,...
797,83,2019-03-17,79.603933,NaN,NaN,NaN
798,83,2019-03-18,80.183401,NaN,NaN,NaN
799,83,2019-03-19,80.961734,NaN,NaN,NaN
800,83,2019-07-25,89.530830,79.62712,NaN,NaN
